### 🧪 Análisis de Datos - Sistema de Ventas Tienda Aurelion
### ETL + Visualizaciones + Análisis para Machine Learning
### **Base de datos:** MariaDB (XAMPP)  
### **Herramientas:** Polars (ETL), Plotly (Visualizaciones), Scikit-learn (ML Prep)

### 1. Importamos las librerias

In [24]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import mysql.connector
from mysql.connector import Error
import warnings
warnings.filterwarnings('ignore')

# Configuración para visualizaciones
import plotly.io as pio
pio.templates.default = "plotly_white"

### 2. Conexion a la base de datos AurelionDB

In [25]:
def conectar_bd():
    """Establece conexión con la base de datos MariaDB en XAMPP"""
    try:
        conexion = mysql.connector.connect(
            host='localhost',
            user='root',  # Usuario por defecto de XAMPP
            password='',  # Password por defecto (vacío en XAMPP)
            database='AurelionDB',
            port=3306  # Puerto por defecto de MySQL en XAMPP
        )
        
        if conexion.is_connected():
            print('✅ Conexión exitosa a la base de datos')
            return conexion
            
    except Error as e:
        print(f'❌ Error al conectar: {e}')
        return None

### 2.1 Probar conexión

In [26]:
conexion = conectar_bd()

✅ Conexión exitosa a la base de datos


### 3. Extracción de datos - POLARS


In [27]:
def extraer_datos_polars(conexion):
    """Extrae todos los datos de la base de datos usando Polars"""
    
    # Consultas SQL para extraer datos
    consultas = {
        'clientes': 'SELECT * FROM Clientes',
        'productos': 'SELECT * FROM Productos', 
        'ventas': 'SELECT * FROM Ventas',
        'detalles_ventas': 'SELECT * FROM Detalles_Ventas'
    }
    
    datos = {}
    
    for nombre, consulta in consultas.items():
        try:
            # Usar Pandas temporalmente para la conexión y luego convertir a Polars
            import pandas as pd
            df_pandas = pd.read_sql(consulta, conexion)
            datos[nombre] = pl.from_pandas(df_pandas)
            print(f'✅ {nombre.capitalize()} extraídos: {len(datos[nombre])} registros')
            
        except Exception as e:
            print(f'❌ Error extrayendo {nombre}: {e}')
    
    return datos


### 3.1 Extraer datos

In [28]:
datos = extraer_datos_polars(conexion)

✅ Clientes extraídos: 100 registros
✅ Productos extraídos: 100 registros
✅ Ventas extraídos: 120 registros
✅ Detalles_ventas extraídos: 343 registros


### 4. Mostrar resumen de datos

In [29]:
for nombre, df in datos.items():
    print(f"{nombre.upper()}: {len(df)} registros, {len(df.columns)} columnas")

CLIENTES: 100 registros, 5 columnas
PRODUCTOS: 100 registros, 4 columnas
VENTAS: 120 registros, 6 columnas
DETALLES_VENTAS: 343 registros, 7 columnas


### 5. Transformación de Datos - POLARS

In [ ]:
def transformar_datos_robusta(datos):
    """Transformación de datos con manejo de errores"""
    
    print("🔄 Iniciando transformaciones...")
    
    # Crear copias
    clientes = datos['clientes'].clone()
    productos = datos['productos'].clone()
    ventas = datos['ventas'].clone()
    detalles = datos['detalles_ventas'].clone()
    
    # 1. Conversiones básicas de tipos
    try:
        detalles = detalles.with_columns([
            pl.col('cantidad').cast(pl.Float64),
            pl.col('importe').cast(pl.Float64),
            pl.col('precio_unitario').cast(pl.Float64)
        ])
        print("✅ Tipos de detalles convertidos")
    except Exception as e:
        print(f"⚠️  Error en detalles: {e}")
    
    try:
        productos = productos.with_columns([
            pl.col('precio_unitario').cast(pl.Float64)
        ])
        print("✅ Tipos de productos convertidos")
    except Exception as e:
        print(f"⚠️  Error en productos: {e}")
    
    # 2. Joins básicos
    try:
        # Join detalles con ventas
        ventas_detalladas = detalles.join(ventas, on='id_venta', how='left')
        # Join con productos
        ventas_detalladas = ventas_detalladas.join(productos, on='id_producto', how='left')
        # Join con clientes
        ventas_detalladas = ventas_detalladas.join(clientes, on='id_cliente', how='left')
        print("✅ Joins completados")
    except Exception as e:
        print(f"❌ Error en joins: {e}")
        ventas_detalladas = detalles  # Usar solo detalles como fallback
    
    # 3. Métricas básicas (si tenemos fecha)
    if 'fecha' in ventas_detalladas.columns:
        try:
            ventas_detalladas = ventas_detalladas.with_columns([
                pl.col('importe').alias('ingreso_total'),
                (pl.col('importe') / pl.col('cantidad')).alias('precio_promedio')
            ])
            print("✅ Métricas básicas calculadas")
        except Exception as e:
            print(f"⚠️  Error en métricas: {e}")
    
    return {
        'ventas_detalladas': ventas_detalladas,
        'datos_originales': datos
    }

# Aplicar transformación
datos_transformados = transformar_datos_robusta(datos)
print(f"📊 Datos transformados: {len(datos_transformados['ventas_detalladas'])} registros")

🔄 Iniciando transformaciones...
✅ Tipos de detalles convertidos
✅ Tipos de productos convertidos
✅ Joins completados
✅ Métricas básicas calculadas
📊 Datos transformados: 343 registros


### 6. VERIFICACIÓN DE DATOS TRANSFORMADOS

In [ ]:
print("🔍 VERIFICACIÓN DE DATOS TRANSFORMADOS:")

vd = datos_transformados['ventas_detalladas']
print(f"Columnas disponibles: {vd.columns}")
print(f"Número de registros: {len(vd)}")
print("\nPrimeras 3 filas:")
print(vd.head(3))

print("\nEstadísticas básicas:")
print(vd.select(['cantidad', 'importe']).describe())

🔍 VERIFICACIÓN DE DATOS TRANSFORMADOS:
Columnas disponibles: ['id_detalle', 'id_venta', 'id_producto', 'nombre_producto', 'cantidad', 'precio_unitario', 'importe', 'fecha', 'id_cliente', 'nombre_cliente', 'email', 'medio_pago', 'nombre_producto_right', 'categoria', 'precio_unitario_right', 'nombre_cliente_right', 'email_right', 'ciudad', 'fecha_alta', 'ingreso_total', 'precio_promedio']
Número de registros: 343

Primeras 3 filas:
shape: (3, 21)
┌────────────┬──────────┬────────────┬────────────┬───┬────────┬───────────┬───────────┬───────────┐
│ id_detalle ┆ id_venta ┆ id_product ┆ nombre_pro ┆ … ┆ ciudad ┆ fecha_alt ┆ ingreso_t ┆ precio_pr │
│ ---        ┆ ---      ┆ o          ┆ ducto      ┆   ┆ ---    ┆ a         ┆ otal      ┆ omedio    │
│ i64        ┆ f64      ┆ ---        ┆ ---        ┆   ┆ str    ┆ ---       ┆ ---       ┆ ---       │
│            ┆          ┆ f64        ┆ str        ┆   ┆        ┆ datetime[ ┆ f64       ┆ f64       │
│            ┆          ┆            ┆        

### 7. VISUALIZACIÓN 1 - DISTRIBUCIÓN DE VENTAS

In [ ]:
print("📈 GENERANDO VISUALIZACIÓN 1...")

vd_pandas = datos_transformados['ventas_detalladas'].to_pandas()

# Histograma de importes
fig1 = px.histogram(vd_pandas, x='importe', 
                   title='Distribución de Importes de Ventas',
                   nbins=30,
                   color_discrete_sequence=['blue'])
fig1.show()

# Boxplot de importes
fig2 = px.box(vd_pandas, y='importe', 
             title='Distribución de Importes (Boxplot)')
fig2.show()

📈 GENERANDO VISUALIZACIÓN 1...


### 8. VISUALIZACIÓN 2 - ANÁLISIS POR PRODUCTO

In [ ]:
print("📈 GENERANDO VISUALIZACIÓN 2...")

# Top 10 productos por ingresos
top_productos = datos_transformados['ventas_detalladas'].group_by('nombre_producto').agg([
    pl.sum('importe').alias('ingresos_totales'),
    pl.sum('cantidad').alias('unidades_vendidas'),
    pl.mean('importe').alias('ticket_promedio')
]).sort('ingresos_totales', descending=True).head(10)

print("🏆 TOP 10 PRODUCTOS:")
print(top_productos)

# Gráfico de barras
fig3 = px.bar(top_productos.to_pandas(), 
             x='nombre_producto', y='ingresos_totales',
             title='Top 10 Productos por Ingresos',
             color='unidades_vendidas',
             labels={'nombre_producto': 'Producto', 'ingresos_totales': 'Ingresos Totales'})
fig3.show()

📈 GENERANDO VISUALIZACIÓN 2...
🏆 TOP 10 PRODUCTOS:
shape: (10, 4)
┌────────────────────────────┬──────────────────┬───────────────────┬─────────────────┐
│ nombre_producto            ┆ ingresos_totales ┆ unidades_vendidas ┆ ticket_promedio │
│ ---                        ┆ ---              ┆ ---               ┆ ---             │
│ str                        ┆ f64              ┆ f64               ┆ f64             │
╞════════════════════════════╪══════════════════╪═══════════════════╪═════════════════╡
│ Desodorante Aerosol        ┆ 93800.0          ┆ 20.0              ┆ 13400.0         │
│ Queso Rallado 150g         ┆ 89544.0          ┆ 26.0              ┆ 9949.333333     │
│ Pizza Congelada Muzzarella ┆ 85720.0          ┆ 20.0              ┆ 17144.0         │
│ Ron 700ml                  ┆ 81396.0          ┆ 21.0              ┆ 11628.0         │
│ Yerba Mate Suave 1kg       ┆ 77560.0          ┆ 20.0              ┆ 12926.666667    │
│ Energética Nitro 500ml     ┆ 71706.0          ┆ 17.0

### 9. VISUALIZACIÓN 3 - ANÁLISIS TEMPORAL

In [ ]:
print("📈 GENERANDO VISUALIZACIÓN 3...")

if 'fecha' in datos_transformados['ventas_detalladas'].columns:
    # Agrupar por mes
    ventas_mensuales = datos_transformados['ventas_detalladas'].with_columns([
        pl.col('fecha').dt.strftime('%Y-%m').alias('mes')
    ]).group_by('mes').agg([
        pl.sum('importe').alias('ingresos_mensuales'),
        pl.count().alias('ventas_totales')
    ]).sort('mes')
    
    fig4 = px.line(ventas_mensuales.to_pandas(), 
                  x='mes', y='ingresos_mensuales',
                  title='Evolución Mensual de Ingresos',
                  markers=True)
    fig4.show()
else:
    print("ℹ️  No hay datos de fecha para análisis temporal")

📈 GENERANDO VISUALIZACIÓN 3...


### 10. VISUALIZACIÓN 4 - ANÁLISIS POR CATEGORÍA

In [ ]:
print("📈 GENERANDO VISUALIZACIÓN 4...")

if 'categoria' in datos_transformados['ventas_detalladas'].columns:
    ventas_categoria = datos_transformados['ventas_detalladas'].group_by('categoria').agg([
        pl.sum('importe').alias('ingresos_totales'),
        pl.sum('cantidad').alias('unidades_vendidas'),
        pl.mean('importe').alias('ticket_promedio')
    ])
    
    # Gráfico de torta
    fig5 = px.pie(ventas_categoria.to_pandas(), 
                 values='ingresos_totales', names='categoria',
                 title='Distribución de Ingresos por Categoría')
    fig5.show()
    
    # Gráfico de barras
    fig6 = px.bar(ventas_categoria.to_pandas(),
                 x='categoria', y='ingresos_totales',
                 title='Ingresos por Categoría',
                 color='unidades_vendidas')
    fig6.show()

📈 GENERANDO VISUALIZACIÓN 4...


### 11. ANÁLISIS DE CLIENTES

In [42]:
print("👥 ANALIZANDO CLIENTES...")

if 'nombre_cliente' in datos_transformados['ventas_detalladas'].columns:
    analisis_clientes = datos_transformados['ventas_detalladas'].group_by('id_cliente', 'nombre_cliente', 'ciudad').agg([
        pl.sum('importe').alias('gasto_total'),
        pl.count().alias('compras_totales'),
        pl.mean('importe').alias('ticket_promedio')
    ]).sort('gasto_total', descending=True)
    
    print("🏆 TOP 10 CLIENTES:")
    print(analisis_clientes.head(10))
    
    # Gráfico de clientes top
    top_clientes = analisis_clientes.head(10)
    fig7 = px.bar(top_clientes.to_pandas(),
                 x='nombre_cliente', y='gasto_total',
                 title='Top 10 Clientes por Gasto Total',
                 color='compras_totales',
                 labels={'nombre_cliente': 'Cliente', 'gasto_total': 'Gasto Total'})
    fig7.show()

👥 ANALIZANDO CLIENTES...
🏆 TOP 10 CLIENTES:
shape: (10, 6)
┌────────────┬────────────────────┬─────────────┬─────────────┬─────────────────┬─────────────────┐
│ id_cliente ┆ nombre_cliente     ┆ ciudad      ┆ gasto_total ┆ compras_totales ┆ ticket_promedio │
│ ---        ┆ ---                ┆ ---         ┆ ---         ┆ ---             ┆ ---             │
│ f64        ┆ str                ┆ str         ┆ f64         ┆ u32             ┆ f64             │
╞════════════╪════════════════════╪═════════════╪═════════════╪═════════════════╪═════════════════╡
│ 5.0        ┆ Agustina Flores    ┆ Cordoba     ┆ 132158.0    ┆ 15              ┆ 8810.533333     │
│ 56.0       ┆ Bruno Diaz         ┆ Rio Cuarto  ┆ 90701.0     ┆ 10              ┆ 9070.1          │
│ 52.0       ┆ Diego Diaz         ┆ Rio Cuarto  ┆ 90522.0     ┆ 11              ┆ 8229.272727     │
│ 25.0       ┆ Karina Castro      ┆ Rio Cuarto  ┆ 81830.0     ┆ 9               ┆ 9092.222222     │
│ 1.0        ┆ Mariana Lopez      ┆ Carlo

### 12. ANÁLISIS DE MÉTODOS DE PAGO

In [43]:
print("💳 ANALIZANDO MÉTODOS DE PAGO...")

if 'medio_pago' in datos_transformados['ventas_detalladas'].columns:
    analisis_pago = datos_transformados['ventas_detalladas'].group_by('medio_pago').agg([
        pl.sum('importe').alias('ingresos_totales'),
        pl.count().alias('transacciones'),
        pl.mean('importe').alias('ticket_promedio')
    ])
    
    print("📋 MÉTODOS DE PAGO:")
    print(analisis_pago)
    
    fig8 = px.bar(analisis_pago.to_pandas(),
                 x='medio_pago', y='ingresos_totales',
                 title='Ingresos por Método de Pago',
                 color='transacciones')
    fig8.show()

💳 ANALIZANDO MÉTODOS DE PAGO...
📋 MÉTODOS DE PAGO:
shape: (4, 4)
┌───────────────┬──────────────────┬───────────────┬─────────────────┐
│ medio_pago    ┆ ingresos_totales ┆ transacciones ┆ ticket_promedio │
│ ---           ┆ ---              ┆ ---           ┆ ---             │
│ str           ┆ f64              ┆ u32           ┆ f64             │
╞═══════════════╪══════════════════╪═══════════════╪═════════════════╡
│ tarjeta       ┆ 460099.0         ┆ 69            ┆ 6668.101449     │
│ qr            ┆ 714280.0         ┆ 91            ┆ 7849.230769     │
│ efectivo      ┆ 934819.0         ┆ 111           ┆ 8421.792793     │
│ transferencia ┆ 542219.0         ┆ 72            ┆ 7530.819444     │
└───────────────┴──────────────────┴───────────────┴─────────────────┘


### 13. PREPARACIÓN PARA MACHINE LEARNING

In [44]:
print("🤖 PREPARANDO DATOS PARA ML...")

# Crear características básicas
datos_ml = datos_transformados['ventas_detalladas'].select([
    'id_venta', 'id_producto', 'id_cliente', 'cantidad', 'importe'
])

# Agregar características si están disponibles
if 'categoria' in datos_transformados['ventas_detalladas'].columns:
    datos_ml = datos_ml.with_columns(
        datos_transformados['ventas_detalladas']['categoria']
    )

if 'medio_pago' in datos_transformados['ventas_detalladas'].columns:
    datos_ml = datos_ml.with_columns(
        datos_transformados['ventas_detalladas']['medio_pago']
    )

print("📊 DATOS PARA ML:")
print(f"Forma: {datos_ml.shape}")
print("Columnas:", datos_ml.columns)
print("\nMuestra:")
print(datos_ml.head(5))

🤖 PREPARANDO DATOS PARA ML...
📊 DATOS PARA ML:
Forma: (343, 7)
Columnas: ['id_venta', 'id_producto', 'id_cliente', 'cantidad', 'importe', 'categoria', 'medio_pago']

Muestra:
shape: (5, 7)
┌──────────┬─────────────┬────────────┬──────────┬─────────┬───────────┬────────────┐
│ id_venta ┆ id_producto ┆ id_cliente ┆ cantidad ┆ importe ┆ categoria ┆ medio_pago │
│ ---      ┆ ---         ┆ ---        ┆ ---      ┆ ---     ┆ ---       ┆ ---        │
│ f64      ┆ f64         ┆ f64        ┆ f64      ┆ f64     ┆ str       ┆ str        │
╞══════════╪═════════════╪════════════╪══════════╪═════════╪═══════════╪════════════╡
│ 1.0      ┆ 90.0        ┆ 62.0       ┆ 1.0      ┆ 2902.0  ┆ Limpieza  ┆ tarjeta    │
│ 2.0      ┆ 82.0        ┆ 49.0       ┆ 5.0      ┆ 11970.0 ┆ Limpieza  ┆ qr         │
│ 2.0      ┆ 39.0        ┆ 49.0       ┆ 5.0      ┆ 2345.0  ┆ Alimentos ┆ qr         │
│ 2.0      ┆ 70.0        ┆ 49.0       ┆ 2.0      ┆ 8122.0  ┆ Limpieza  ┆ qr         │
│ 2.0      ┆ 22.0        ┆ 49.0      

### 14. EXPORTACIÓN DE DATOS

In [ ]:
print("💾 EXPORTANDO DATOS...")

import os
os.makedirs('datos_exportados', exist_ok=True)

# Exportar datos procesados
datos_transformados['ventas_detalladas'].write_csv('datos_exportados/ventas_procesadas.csv')

# Exportar análisis específicos
if 'analisis_clientes' in datos_transformados:
    datos_transformados['analisis_clientes'].write_csv('datos_exportados/analisis_clientes.csv')

print("✅ Datos exportados en la carpeta 'datos_exportados/'")

💾 EXPORTANDO DATOS...
✅ Datos exportados en la carpeta 'datos_exportados/'


### 15. RESUMEN FINAL

In [ ]:
print("\n" + "="*60)
print("🎉 ANÁLISIS COMPLETADO EXITOSAMENTE")
print("="*60)

vd = datos_transformados['ventas_detalladas']
print(f"📊 Total registros procesados: {len(vd)}")
print(f"💰 Ingreso total: {vd['importe'].sum():,.2f}")
print(f"📦 Transacciones únicas: {vd['id_venta'].n_unique()}")

if 'nombre_cliente' in vd.columns:
    print(f"👥 Clientes únicos: {vd['id_cliente'].n_unique()}")

if 'nombre_producto' in vd.columns:
    print(f"🏪 Productos únicos: {vd['id_producto'].n_unique()}")

print("📈 Visualizaciones generadas: 7+ gráficos interactivos")
print("💾 Datos exportados para análisis futuros")
print("🤖 Datos preparados para Machine Learning")
print("="*60)

# Cerrar conexión
if conexion and conexion.is_connected():
    conexion.close()
    print("✅ Conexión a BD cerrada")


🎉 ANÁLISIS COMPLETADO EXITOSAMENTE
📊 Total registros procesados: 343
💰 Ingreso total: 2,651,417.00
📦 Transacciones únicas: 120
👥 Clientes únicos: 67
🏪 Productos únicos: 95
📈 Visualizaciones generadas: 7+ gráficos interactivos
💾 Datos exportados para análisis futuros
🤖 Datos preparados para Machine Learning
✅ Conexión a BD cerrada


# Anexos

In [47]:
# =============================================================================
# 16. FUNCIÓN PARA CONSULTAS RÁPIDAS (OPCIONAL)
# =============================================================================
def consulta_rapida(sql_query):
    """Función para ejecutar consultas SQL rápidas"""
    conexion = conectar_bd()
    if conexion:
        import pandas as pd
        resultado = pd.read_sql(sql_query, conexion)
        conexion.close()
        return pl.from_pandas(resultado)
    return None

# Ejemplo de uso:
resultado = consulta_rapida("SELECT * FROM Ventas LIMIT 5")
print(resultado)

✅ Conexión exitosa a la base de datos
shape: (5, 6)
┌──────────┬──────────────┬────────────┬──────────────────┬────────────────────────┬───────────────┐
│ id_venta ┆ fecha        ┆ id_cliente ┆ nombre_cliente   ┆ email                  ┆ medio_pago    │
│ ---      ┆ ---          ┆ ---        ┆ ---              ┆ ---                    ┆ ---           │
│ f64      ┆ datetime[ns] ┆ f64        ┆ str              ┆ str                    ┆ str           │
╞══════════╪══════════════╪════════════╪══════════════════╪════════════════════════╪═══════════════╡
│ 1.0      ┆ 2024-06-19   ┆ 62.0       ┆ Guadalupe Romero ┆ guadalupe.romero@mail. ┆ tarjeta       │
│          ┆ 00:00:00     ┆            ┆                  ┆ com                    ┆               │
│ 2.0      ┆ 2024-03-17   ┆ 49.0       ┆ Olivia Gomez     ┆ olivia.gomez@mail.com  ┆ qr            │
│          ┆ 00:00:00     ┆            ┆                  ┆                        ┆               │
│ 3.0      ┆ 2024-01-13   ┆ 20.0       